In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np
from scipy.stats import mode
import csv
import matplotlib.dates
from datetime import *
from sklearn import linear_model
from sklearn.preprocessing import *
from sklearn import ensemble
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.externals import joblib
import xgboost as xgb
from sklearn import metrics
from xgboost.sklearn import XGBClassifier  
from sklearn.model_selection import GridSearchCV,cross_val_score  
import matplotlib.pylab as plt 

In [279]:
#首先导入滑窗训练集数据进行线下数据集划分
train_df = pd.read_csv('chusai_huachuang_2daysHistory_train_df.csv')
test_df = pd.read_csv('chusai_huachuang_2daysHistory_test_df.csv')

fea = [
#         'context_page_id_1', 
#         'context_page_id_2', 'context_page_id_3', 'context_page_id_4', 
       'item_pv_level_low', 'shop_star_level_loss', 
        'user_gender_id', 'user_occupation_id', 'user_star_level',
       'hour', 'day_hour', #'weekday',
       'user_age_level', 'item_sales_level', 'item_brand_id', 'item_city_id',
       'item_collected_level', 'item_price_level', 'item_pv_level',
       'context_page_id', 'shop_review_num_level', 'shop_star_level',
#        'match_category_proportion', 
#         'match_property_proportion',
#        'predict_category_number', 'predict_property_number', 
        'category_number', 
        'property_number', 
#        'isFirstCategoryIn', 'isLastCategoryIn', 
       
        'lastOneHour_sameItem_count', 'lastOneHour_sameFirstCategory_count', 
        'lastOneHour_sameLastCategory_count', 'lastOneHour_sameBrand_count', 'lastOneHour_sameShop_count', 
    
        'lastOneDay_sameItem_count', 'lastOneDay_sameFirstCategory_count', 'lastOneDay_sameLastCategory_count', 
       'lastOneDay_sameBrand_count', 'lastOneDay_sameShop_count', 
        
#        'isLastOneHour_firstClickItem', 
       'is_history_sameItem', 'is_history_sameFirstCategory', 'is_history_sameLastCategory',
       'is_highProportion_brand', 'is_highSale_brand', 
       'isHigh_shop_score_service', 'isHigh_shop_score_delivery',
#        'isHigh_shop_review_positive_rate', 'isHigh_shop_score_description', 
       'is_history_sameBrand', 'is_history_sameShop', 
#         'is_history_firstClickItem',
#        'shop_review_positive_rate_diff', 'shop_score_service_diff', 'shop_score_delivery_diff', 'shop_score_description_diff', 
        'shop_review_positive_rate', 'shop_score_service', 'shop_score_delivery', 'shop_score_description',
#        'shop_item_classNumber', 'brand_item_classNumber', 'city_item_classNumber', 
#        'shop_user_classNumber', 'brand_user_classNumber', 'city_user_classNumber',
       
        'hour_converse_smooth_rate',
    
        'user_id_total_number', 'item_brand_id_total_number', 'item_id_total_number', 
       'shop_id_total_number', 'item_city_id_total_number', 
       'real_first_category_total_number', 'real_last_category_total_number', 
#        'user_item_total_number', 'user_brand_total_number', 'user_shop_total_number',
#         'user_city_total_number', 'user_first_category_total_number', 'user_last_category_total_number',
        
        'user_id_buy_number', 'item_brand_id_buy_number', 'item_id_buy_number', 
       'shop_id_buy_number', 'item_city_id_buy_number', 
       'real_first_category_buy_number', 'real_last_category_buy_number', 
#        'user_item_buy_number', 'user_brand_buy_number', 'user_shop_buy_number',
#         'user_city_buy_number', 'user_first_category_buy_number', 'user_last_category_buy_number', 
        
        'user_id_converse_smooth_rate', 'item_brand_id_converse_smooth_rate', 'item_id_converse_smooth_rate', 
       'shop_id_converse_smooth_rate', 'item_city_id_converse_smooth_rate', 
       'real_first_category_converse_smooth_rate', 'real_last_category_converse_smooth_rate', 
#        'user_item_converse_smooth_rate', 'user_brand_converse_smooth_rate', 'user_shop_converse_smooth_rate',
#         'user_city_converse_smooth_rate', 'user_first_category_converse_smooth_rate', 'user_last_category_converse_smooth_rate', 
       
#         'user_item_price_converse_smooth_rate', 'user_item_sales_converse_smooth_rate', 
#         'user_item_price_total_number', 'user_item_sales_total_number',
#         'user_item_price_buy_number', 'user_item_sales_buy_number',
        
#         'userItem_lastClickDeltaTime', 
#         'userBrand_lastClickDeltaTime', 'userShop_lastClickDeltaTime', 
#         'userFirstCategory_lastClickDeltaTime', 'userLastCategory_lastClickDeltaTime', 
    
#     'user_gender_item_converse_smooth_rate', 'user_gender_brand_converse_smooth_rate', 'user_gender_shop_converse_smooth_rate', 
#     'user_age_item_converse_smooth_rate', 'user_age_brand_converse_smooth_rate', 'user_age_shop_converse_smooth_rate', 
#     'user_occupation_item_converse_smooth_rate', 'user_occupation_brand_converse_smooth_rate', 'user_occupation_shop_converse_smooth_rate', 
#     'user_star_item_converse_smooth_rate', 'user_star_brand_converse_smooth_rate', 'user_star_shop_converse_smooth_rate', 
    
#     'user_gender_item_total_number', 'user_gender_brand_total_number', 'user_gender_shop_total_number', 
#     'user_age_item_total_number', 'user_age_brand_total_number', 'user_age_shop_total_number', 
#     'user_occupation_item_total_number', 'user_occupation_brand_total_number', 'user_occupation_shop_total_number',
#     'user_star_item_total_number', 'user_star_brand_total_number', 'user_star_shop_total_number', 
    
#      'user_gender_item_buy_number', 'user_gender_brand_buy_number', 'user_gender_shop_buy_number', 
#       'user_age_item_buy_number', 'user_age_brand_buy_number', 'user_age_shop_buy_number', 
#     'user_occupation_item_buy_number', 'user_occupation_brand_buy_number', 'user_occupation_shop_buy_number',
#     'user_star_item_buy_number', 'user_star_brand_buy_number',  'user_star_shop_buy_number',
    
        'history_sameItem_count', 'history_sameFirstCategory_count', 'history_sameLastCategory_count',
        'history_sameBrand_count', 'history_sameShop_count'
      ]

train_dataset1_x = train_df[fea][(train_df.day >= 20) & (train_df.day <= 23)]
train_dataset1_y = train_df['is_trade'][(train_df.day >= 20) & (train_df.day <= 23)]
test_dataset1_x = train_df[fea][(train_df.day == 24)]
test_dataset1_y = train_df['is_trade'][(train_df.day == 24)]

train_dataset_x = train_df[fea][(train_df.day >= 20) & (train_df.day <= 24)]
train_dataset_y = train_df['is_trade'][(train_df.day >= 20) & (train_df.day <= 24)]
test_dataset_x = test_df[fea]
print(len(train_dataset_x.columns))
print(len(test_dataset_x.columns))


69
69


In [282]:
#开始进行xgb模型训练
xgb_train_dataset1 = xgb.DMatrix(train_dataset1_x,label = train_dataset1_y)
xgb_test_dataset1 = xgb.DMatrix(test_dataset1_x, label = test_dataset1_y)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.5,
        'min_child_weight':10,
        'max_depth':3,
        'lambda':11,
        'subsample':0.85,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.1,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

offline_watchlist = [(xgb_train_dataset1,'train'), (xgb_test_dataset1, 'val')]
offline_model = xgb.train(cv_params, xgb_train_dataset1, num_boost_round=5000, evals=offline_watchlist)


[0]	train-logloss:0.605278	val-logloss:0.604627
[1]	train-logloss:0.533411	val-logloss:0.531984
[2]	train-logloss:0.473574	val-logloss:0.471677
[3]	train-logloss:0.423139	val-logloss:0.420575
[4]	train-logloss:0.380235	val-logloss:0.377219
[5]	train-logloss:0.343448	val-logloss:0.339919
[6]	train-logloss:0.311711	val-logloss:0.30786
[7]	train-logloss:0.284211	val-logloss:0.280025
[8]	train-logloss:0.26032	val-logloss:0.255669
[9]	train-logloss:0.239437	val-logloss:0.234452
[10]	train-logloss:0.221185	val-logloss:0.215775
[11]	train-logloss:0.205164	val-logloss:0.199522
[12]	train-logloss:0.191103	val-logloss:0.185257
[13]	train-logloss:0.178737	val-logloss:0.172728
[14]	train-logloss:0.167852	val-logloss:0.161707
[15]	train-logloss:0.1583	val-logloss:0.152549
[16]	train-logloss:0.149844	val-logloss:0.143897
[17]	train-logloss:0.142416	val-logloss:0.136152
[18]	train-logloss:0.135868	val-logloss:0.129308
[19]	train-logloss:0.130112	val-logloss:0.123441
[20]	train-logloss:0.125034	val-lo

[167]	train-logloss:0.087272	val-logloss:0.082223
[168]	train-logloss:0.087266	val-logloss:0.082222
[169]	train-logloss:0.087249	val-logloss:0.082238
[170]	train-logloss:0.087233	val-logloss:0.082228
[171]	train-logloss:0.087214	val-logloss:0.082227
[172]	train-logloss:0.0872	val-logloss:0.082264
[173]	train-logloss:0.087191	val-logloss:0.082263
[174]	train-logloss:0.087174	val-logloss:0.08225
[175]	train-logloss:0.087159	val-logloss:0.082237
[176]	train-logloss:0.087149	val-logloss:0.082241
[177]	train-logloss:0.087135	val-logloss:0.082243
[178]	train-logloss:0.087122	val-logloss:0.082211
[179]	train-logloss:0.087114	val-logloss:0.082211
[180]	train-logloss:0.087094	val-logloss:0.082207
[181]	train-logloss:0.087085	val-logloss:0.082197
[182]	train-logloss:0.087076	val-logloss:0.082139
[183]	train-logloss:0.087071	val-logloss:0.082137
[184]	train-logloss:0.087055	val-logloss:0.082117
[185]	train-logloss:0.087038	val-logloss:0.082114
[186]	train-logloss:0.08703	val-logloss:0.082116
[187

[332]	train-logloss:0.085628	val-logloss:0.082032
[333]	train-logloss:0.085618	val-logloss:0.082032
[334]	train-logloss:0.085607	val-logloss:0.082005
[335]	train-logloss:0.0856	val-logloss:0.081995
[336]	train-logloss:0.085598	val-logloss:0.081942
[337]	train-logloss:0.085586	val-logloss:0.081915
[338]	train-logloss:0.085578	val-logloss:0.081939
[339]	train-logloss:0.085564	val-logloss:0.081901
[340]	train-logloss:0.085555	val-logloss:0.08188
[341]	train-logloss:0.085539	val-logloss:0.081882
[342]	train-logloss:0.08553	val-logloss:0.08189
[343]	train-logloss:0.085521	val-logloss:0.08189
[344]	train-logloss:0.085516	val-logloss:0.081884
[345]	train-logloss:0.085507	val-logloss:0.081882
[346]	train-logloss:0.0855	val-logloss:0.081906
[347]	train-logloss:0.08549	val-logloss:0.081882
[348]	train-logloss:0.08548	val-logloss:0.081864
[349]	train-logloss:0.085467	val-logloss:0.081836
[350]	train-logloss:0.085457	val-logloss:0.081844
[351]	train-logloss:0.085453	val-logloss:0.081843
[352]	trai

[497]	train-logloss:0.08429	val-logloss:0.081903
[498]	train-logloss:0.084282	val-logloss:0.081901
[499]	train-logloss:0.084274	val-logloss:0.081896
[500]	train-logloss:0.084263	val-logloss:0.081895
[501]	train-logloss:0.084253	val-logloss:0.081888
[502]	train-logloss:0.084242	val-logloss:0.08189
[503]	train-logloss:0.084237	val-logloss:0.081887
[504]	train-logloss:0.084229	val-logloss:0.081887
[505]	train-logloss:0.084226	val-logloss:0.081893
[506]	train-logloss:0.084219	val-logloss:0.081886
[507]	train-logloss:0.084215	val-logloss:0.081886
[508]	train-logloss:0.084205	val-logloss:0.081889
[509]	train-logloss:0.084194	val-logloss:0.081885
[510]	train-logloss:0.084187	val-logloss:0.081888
[511]	train-logloss:0.084178	val-logloss:0.081901
[512]	train-logloss:0.084172	val-logloss:0.081919
[513]	train-logloss:0.084165	val-logloss:0.081922
[514]	train-logloss:0.084157	val-logloss:0.081926
[515]	train-logloss:0.08415	val-logloss:0.081924
[516]	train-logloss:0.084144	val-logloss:0.081922
[51

KeyboardInterrupt: 

In [51]:
#输出线下训练的xgb模型的重要性
feat_imp = pd.Series(offline_model.get_fscore()).sort_values(ascending=False)
print(feat_imp.head(100))


hour                                        52
shop_score_delivery                         51
shop_id_converse_smooth_rate                48
item_id_converse_smooth_rate                46
item_sales_level                            45
user_star_level                             40
item_brand_id_converse_smooth_rate          39
user_age_level                              32
is_highProportion_brand                     29
item_price_level                            29
real_first_category_converse_smooth_rate    28
item_brand_id_total_number                  27
lastOneDay_sameItem_count                   27
shop_score_service                          24
shop_score_description_diff                 24
item_city_id_converse_smooth_rate           22
match_property_proportion                   21
shop_score_description                      20
shop_id_total_number                        19
context_page_id                             19
history_sameItem_count                      18
isFirstCatego

In [52]:
#计算线下误差
def calError(col1, col2):
    error = metrics.log_loss(col1, col2)
    return error

xgb_test = xgb.DMatrix(test_dataset1_x)
test_result = offline_model.predict(xgb_test)
dataset1_error = calError(test_dataset1_y.values, test_result)
print('dataset1_error : ', dataset1_error)

dataset1_error :  0.0818752857022


In [264]:
#进行正式模型训练
xgb_train = xgb.DMatrix(train_dataset_x, label=train_dataset_y)
xgb_test = xgb.DMatrix(test_dataset_x)

cv_params={'booster':'gbtree',
        'objective': 'binary:logistic',
        'eval_metric':'logloss',
        'gamma':0.5,
        'min_child_weight':10,
        'max_depth':3,
        'lambda':11,
        'subsample':0.85,
        'colsample_bytree':0.7,
        'colsample_bylevel':0.7,
        'eta': 0.1,
        'tree_method':'exact',
        'seed':0,
        'nthread':12
        }

watchlist = [(xgb_train,'train')]
model = xgb.train(cv_params, xgb_train, num_boost_round=310, evals=watchlist)


[0]	train-logloss:0.604999
[1]	train-logloss:0.532855
[2]	train-logloss:0.472826
[3]	train-logloss:0.422219
[4]	train-logloss:0.379151
[5]	train-logloss:0.342245
[6]	train-logloss:0.310373
[7]	train-logloss:0.28277
[8]	train-logloss:0.258776
[9]	train-logloss:0.237815
[10]	train-logloss:0.219463
[11]	train-logloss:0.203393
[12]	train-logloss:0.189277
[13]	train-logloss:0.176853
[14]	train-logloss:0.165909
[15]	train-logloss:0.156281
[16]	train-logloss:0.14779
[17]	train-logloss:0.140343
[18]	train-logloss:0.133784
[19]	train-logloss:0.12799
[20]	train-logloss:0.122914
[21]	train-logloss:0.118419
[22]	train-logloss:0.114472
[23]	train-logloss:0.11101
[24]	train-logloss:0.107985
[25]	train-logloss:0.105316
[26]	train-logloss:0.102985
[27]	train-logloss:0.100959
[28]	train-logloss:0.099178
[29]	train-logloss:0.097619
[30]	train-logloss:0.096282
[31]	train-logloss:0.095103
[32]	train-logloss:0.094081
[33]	train-logloss:0.093194
[34]	train-logloss:0.092425
[35]	train-logloss:0.091758
[36]	t

[288]	train-logloss:0.083807
[289]	train-logloss:0.083801
[290]	train-logloss:0.08379
[291]	train-logloss:0.083782
[292]	train-logloss:0.083774
[293]	train-logloss:0.083772
[294]	train-logloss:0.083762
[295]	train-logloss:0.083755
[296]	train-logloss:0.083746
[297]	train-logloss:0.083735
[298]	train-logloss:0.083727
[299]	train-logloss:0.083719
[300]	train-logloss:0.083708
[301]	train-logloss:0.083695
[302]	train-logloss:0.083687
[303]	train-logloss:0.083677
[304]	train-logloss:0.083665
[305]	train-logloss:0.083658
[306]	train-logloss:0.083651
[307]	train-logloss:0.083645
[308]	train-logloss:0.083636
[309]	train-logloss:0.083627


In [265]:
#predict test set
test_df['predicted_score'] = model.predict(xgb_test)


In [266]:
print(len(test_df[['instance_id', 'predicted_score']][test_df.predicted_score > 0.1]))
print(test_df['predicted_score'].mean())
print(len(test_df['predicted_score']))


91
0.016429917886853218
42888


In [60]:
# 导出预测结果
def exportResult(df, fileName):
    df.to_csv('../../data/chusai/submit/%s.txt' % fileName, sep=' ', header=True, index=False)

print(test_df[['instance_id', 'predicted_score']].head(10))
exportResult(test_df[['instance_id', 'predicted_score']], 'chusai_xgb_4_19_wen')


        instance_id  predicted_score
0    55144604125888         0.015501
1   221669167755726         0.005934
2   566644865989395         0.031981
3   954943998950521         0.007564
4  1192015136416062         0.016803
5  1239632009133923         0.006971
6  1370758425798224         0.014562
7  1386245957915884         0.011186
8  1587829942816220         0.008875
9  1965296257027358         0.007834
